# Mod 4 Project - Starter Notebook

This notebook has been provided to you so that you can make use of the following starter code to help with the trickier parts of preprocessing the Zillow dataset. 

The notebook contains a rough outline the general order you'll likely want to take in this project. You'll notice that most of the areas are left blank. This is so that it's more obvious exactly when you should make use of the starter code provided for preprocessing. 

**_NOTE:_** The number of empty cells are not meant to infer how much or how little code should be involved in any given step--we've just provided a few for your convenience. Add, delete, and change things around in this notebook as needed!

# Some Notes Before Starting

This project will be one of the more challenging projects you complete in this program. This is because working with Time Series data is a bit different than working with regular datasets. In order to make this a bit less frustrating and help you understand what you need to do (and when you need to do it), we'll quickly review the dataset formats that you'll encounter in this project. 

## Wide Format vs Long Format

If you take a look at the format of the data in `zillow_data.csv`, you'll notice that the actual Time Series values are stored as separate columns. Here's a sample: 

<img src='~/../images/df_head.png'>

You'll notice that the first seven columns look like any other dataset you're used to working with. However, column 8 refers to the median housing sales values for April 1996, column 9 for May 1996, and so on. This This is called **_Wide Format_**, and it makes the dataframe intuitive and easy to read. However, there are problems with this format when it comes to actually learning from the data, because the data only makes sense if you know the name of the column that the data can be found it. Since column names are metadata, our algorithms will miss out on what dates each value is for. This means that before we pass this data to our ARIMA model, we'll need to reshape our dataset to **_Long Format_**. Reshaped into long format, the dataframe above would now look like:

<img src='~/../images/melted1.png'>

There are now many more rows in this dataset--one for each unique time and zipcode combination in the data! Once our dataset is in this format, we'll be able to train an ARIMA model on it. The method used to convert from Wide to Long is `pd.melt()`, and it is common to refer to our dataset as 'melted' after the transition to denote that it is in long format. 

# Helper Functions Provided

Melting a dataset can be tricky if you've never done it before, so you'll see that we have provided a sample function, `melt_data()`, to help you with this step below. Also provided is:

* `get_datetimes()`, a function to deal with converting the column values for datetimes as a pandas series of datetime objects
* Some good parameters for matplotlib to help make your visualizations more readable. 

Good luck!


# Step 1: Load the Data/Filtering for Chosen Zipcodes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import itertools
import statsmodels.api as sm
from matplotlib.pylab import rcParams
from zip_code_funct import ZipCode_TSA
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
df = pd.read_csv('zillow_data.csv')

In [3]:
all_zipcodes = list(df['RegionName'])

In [4]:
def melt_data(df):
    melted = pd.melt(df, id_vars=['RegionName', 'City', 'State', 'CountyName'], var_name='time')
    melted['time'] = pd.to_datetime(melted['time'], infer_datetime_format=True)
    melted = melted.dropna(subset=['value'])
    return melted

df = pd.read_csv('zillow_data.csv')

df = df.drop(labels=['SizeRank','Metro', 'RegionID'], axis = 1)

df_long = melt_data(df)


df_long.rename(columns={'RegionName' : 'ZipCode'},inplace=True)

Refactor into function to get all of them to compare.

In [ ]:
rk_1_2011.plot(y= 'value', figsize=(12,8), label= '@60657')
plt.legend()
plt.ylabel("Cost of Property")
plt.show();

p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
pdqs = [(x[0], x[1], x[2], 1) for x in list(itertools.product(p, d, q))]


ans = []
for comb in pdq:
    for combs in pdqs:
        try:
            mod = sm.tsa.statespace.SARIMAX(rk_1_2011['value'],
                                            order=comb,
                                            seasonal_order=combs,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            output = mod.fit()
            ans.append([comb, combs, output.aic])
            print('ARIMA {} x {}12 : AIC Calculated ={}'.format(comb, combs, output.aic))
        except:
            continue

ans_df = pd.DataFrame(ans, columns=['pdq', 'pdqs', 'aic'])
ans_df.loc[ans_df['aic'].idxmin()]

ARIMA_MODEL = sm.tsa.statespace.SARIMAX(rk_1_2011['value'],
                                order=(1, 1, 1),
                                seasonal_order=(1, 1, 1, 1),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

output = ARIMA_MODEL.fit()

print(output.summary().tables[1])


output.plot_diagnostics(figsize=(15, 18))
plt.show()



pred = output.get_prediction(start=pd.to_datetime('2016-01-01'), dynamic=False)
pred_conf = pred.conf_int()

rcParams['figure.figsize'] = 15, 6

#Plot observed values
ax = rk_1_2011['value']['2011':].plot(label='observed')

#Plot predicted values
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.9)

#Plot the range for confidence intervals
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='g', alpha=.5)

#Set axes labels
ax.set_xlabel('Date')
ax.set_ylabel('Price')
plt.legend()

plt.show()

# Get forecast 6 steps ahead in future
prediction = output.get_forecast(steps=6)

# Get confidence intervals of forecasts
pred_conf = prediction.conf_int()

ax = rk_1_2011['value'].plot(label='observed', figsize=(20, 15))
prediction.predicted_mean.plot(ax=ax, label='Forecast')
ax.fill_between(pred_conf.index,
                pred_conf.iloc[:, 0],
                pred_conf.iloc[:, 1], color='k', alpha=.25)
ax.set_xlabel('Date')
ax.set_ylabel('Price')

plt.legend()
plt.show()

# Step 6: Interpreting Results

In [ ]:
# zipcode_77494 = ZipCode_TSA(df_long, 77494, '2011')
(77494, zipcode_77494.forecast())


In [11]:
result_list = []
for zipcode in all_zipcodes[:1]:
    predict_conf = temp.prediction_conf()
    temp = ZipCode_TSA(df_long, zipcode, '2011')
    result = temp.forecast() 
    max_gain = predict_conf[1].iloc[:, 1] - predict_conf['value'][-1]
    max_loss = predict_conf[1].iloc[:, 0] - predict_conf['value'][-1]
    result_list.append((zipcode, result, max_gain[-1], max_loss[-1]))
result_list

ValueError: arrays must all be same length

In [57]:
test = ZipCode_TSA(df_long, all_zipcodes[1], "2011")

In [52]:
test.pdq()

[{'comb': (2, 2, 2), 'combs': (2, 2, 2, 1), 'aic': 1251.108387690939}]

In [58]:
test.forecast()

AttributeError: 'tuple' object has no attribute 'predicted_mean'

In [ ]:
import pickle
with open(".pkl", "wb") as f:
    pickle.dump(result_list, f)

In [ ]:
with open('.pkl', rb) as f:
    result_list = pickle.load(f)

In [19]:
aic_dict= _

In [21]:
df = pd.DataFrame(aic_dict)

In [23]:
df['aic'].min()

1544.6331378895513

In [ ]:
{'comb': (1, 1, 1), 'combs': (1, 1, 1, 1),

In [ ]:
sm.tsa.statespace.SARIMAX(df,
                            order=comb,
                            seasonal_order=combs,
                            enforce_stationarity=False,
                            enforce_invertibility=False)

In [ ]:
first_100_df = pd.DataFrame(result_list, columns=['ZipCode','Mean','Max_gain','Max_loss'])

In [ ]:
first_100_df.head()

In [ ]:
result_list_101_1500 = []
for zipcode in all_zipcodes[101:1500]:
    temp = ZipCode_TSA(df_long, zipcode, '2011')
    result = temp.forecast() 
    max_gain = temp.prediction_conf()[1].iloc[:, 1] - temp.df_zipcode()['value'][-1]
    max_loss = temp.prediction_conf()[1].iloc[:, 0] - temp.df_zipcode()['value'][-1]
    result_list.append((zipcode, result, max_gain[-1], max_loss[-1]))


##RUN THE BELOW IF THE ABOVE CELL COMPLETES

In [ ]:
df_101_1500 = pd.DataFrame(result_list_101_1500, columns=['ZipCode','Mean','Max_gain','Max_loss'])

In [ ]:
temp.df_zipcode()['value'][-1]

In [ ]:
import math

# the range of possible gains or losses
abs(pred_conf.iloc[:, 0] - pred_conf.iloc[:, 1])

#What the forecast is at the end of 6 months
prediction.predicted_mean[-1] - prediction.predicted_mean[0]

prediction.predicted_mean

# predicted max gains
max_gain = pred_conf.iloc[:, 1] - rk_1_2011['value'][-1]

# Predicted max losses
max_loss = pred_conf.iloc[:, 0] - rk_1_2011['value'][-1]

(max_loss)/rk_1_2011['value'][-1]